In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

## Pobranie danych

In [2]:
import os
path = os.path.join("datasets", "dielectron.csv")

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(path)
plus_data = pd.DataFrame(data.loc[data["Q2"] >= 0])
minus_data = pd.DataFrame(data.loc[data["Q2"] < 0])
data = pd.concat([plus_data.head(1000), minus_data.head(1000)], axis=0)
data = data.sample(frac=1).reset_index(drop=True)

Ze względu na to, że zbiór jest czysty to sam go zanieczyszczam (usuwając losowe wartości) aby potem go wyczyścić z NaNów

In [4]:
print(data.Q1.isnull().values.any())
print(data.phi1.isnull().values.any())
print(data.E2.isnull().values.any())

False
False
False


In [5]:
import random

def make_nan(x):
    if random.randint(0, 100) < 100:
        return x
    return np.NaN

data.Q1 = data.Q1.apply(lambda x: make_nan(x))
data.phi1 = data.phi1.apply(lambda x: make_nan(x))
data.E2 = data.E2.apply(lambda x: make_nan(x))

In [6]:
print(data.Q1.isnull().values.any())
print(data.phi1.isnull().values.any())
print(data.E2.isnull().values.any())

True
True
True


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Run     2000 non-null   int64  
 1   Event   2000 non-null   int64  
 2   E1      2000 non-null   float64
 3   px1     2000 non-null   float64
 4   py1     2000 non-null   float64
 5   pz1     2000 non-null   float64
 6   pt1     2000 non-null   float64
 7   eta1    2000 non-null   float64
 8   phi1    1977 non-null   float64
 9   Q1      1967 non-null   float64
 10  E2      1977 non-null   float64
 11  px2     2000 non-null   float64
 12  py2     2000 non-null   float64
 13  pz2     2000 non-null   float64
 14  pt2     2000 non-null   float64
 15  eta2    2000 non-null   float64
 16  phi2    2000 non-null   float64
 17  Q2      2000 non-null   int64  
 18  M       2000 non-null   float64
dtypes: float64(16), int64(3)
memory usage: 297.0 KB


Teraz zamienię inty na stringi żeby potem je naprawić

In [8]:
data.Run = data.Run.astype(str)

Sprawdzam jak wyglądają dane

In [9]:
data.head()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
0,147115,368546272,9.34051,-2.015710,0.520275,-9.10556,2.08177,-2.181630,2.889000,1.0,19.4326,-11.80940,-9.27687,-12.3330,15.0174,-0.749175,-2.475720,-1,10.02330
1,147115,14990913,5.20570,4.572420,-1.138860,2.21254,4.71212,0.453805,-0.244105,1.0,103.7640,-18.48930,-8.89345,101.7160,20.5170,2.304090,-2.693260,1,27.91160
2,147115,482260690,4.41252,-0.974871,-2.007580,3.80651,2.23176,1.303660,-2.022850,-1.0,90.8150,6.10354,15.58020,89.2601,16.7331,2.375980,1.197420,-1,14.01300
3,147115,166685143,14.12550,-0.440379,-5.169440,-13.13820,5.18817,-1.659180,-1.655780,-1.0,52.9553,-13.36680,-13.59660,-49.4037,19.0667,-1.680540,-2.347670,1,6.74841
4,147115,482102034,47.78880,-14.024100,-13.218200,43.73070,19.27160,1.557920,-2.385770,-1.0,12.2436,3.08141,1.28393,11.7798,3.3382,1.973600,0.394792,-1,16.13420


In [10]:
data.tail()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
1995,147115,262763738,31.42320,-4.81677,18.48960,24.94690,19.10670,1.081900,1.825640,1.0,2.87905,0.918533,-1.97035,1.88757,2.17393,0.785099,-1.13457,1,12.97960
1996,147115,303068443,24.95120,15.00330,-4.27880,19.47190,15.60150,1.046390,-0.277815,-1.0,10.31190,-3.944290,-3.72314,-8.77021,5.42395,-1.257930,-2.38503,1,30.70220
1997,147115,414734899,98.13490,-31.40010,13.97930,-91.91890,34.37130,-1.710090,2.722740,-1.0,4.98165,-1.224380,1.22865,-4.66992,1.73456,-1.716370,2.35446,1,2.82803
1998,147115,403427630,2.66969,-0.59581,-1.92725,1.74871,2.01725,0.784043,-1.870630,-1.0,112.91900,8.054460,22.78790,110.30200,24.16950,2.223070,1.23105,1,17.73640
1999,147115,373027382,29.49230,13.19170,-12.88570,-23.01590,18.44080,-1.046410,-0.773666,-1.0,17.25330,-3.518780,-1.82520,-16.79180,3.96398,-2.150440,-2.66310,1,17.04470


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Run     2000 non-null   object 
 1   Event   2000 non-null   int64  
 2   E1      2000 non-null   float64
 3   px1     2000 non-null   float64
 4   py1     2000 non-null   float64
 5   pz1     2000 non-null   float64
 6   pt1     2000 non-null   float64
 7   eta1    2000 non-null   float64
 8   phi1    1977 non-null   float64
 9   Q1      1967 non-null   float64
 10  E2      1977 non-null   float64
 11  px2     2000 non-null   float64
 12  py2     2000 non-null   float64
 13  pz2     2000 non-null   float64
 14  pt2     2000 non-null   float64
 15  eta2    2000 non-null   float64
 16  phi2    2000 non-null   float64
 17  Q2      2000 non-null   int64  
 18  M       2000 non-null   float64
dtypes: float64(16), int64(2), object(1)
memory usage: 297.0+ KB


In [12]:
data.describe()

,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
count,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,1977.000000,1967.000000,1977.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000
mean,3.275772e+08,32.583646,0.362022,0.238989,-0.845287,12.407287,-0.038423,0.068352,0.023894,40.343679,-0.013632,0.224399,-0.707894,12.008478,-0.089447,0.026183,0.00000,27.460362
std,1.508022e+08,35.885805,11.160851,11.887759,45.642403,10.585552,1.494036,1.778407,0.999969,41.717796,11.732437,11.032871,55.852992,10.730759,1.773803,1.832277,1.00025,23.335679
min,9.500362e+06,0.834900,-56.895300,-73.517500,-253.916000,0.398498,-2.585940,-3.133880,-1.000000,0.472500,-111.342000,-62.579700,-298.311000,0.277460,-2.586640,-3.141400,-1.00000,2.004170
25%,2.194487e+08,7.633027,-4.173105,-4.574718,-13.365825,3.504293,-1.311073,-1.472760,-1.000000,10.583300,-4.814035,-3.923737,-20.914850,3.482925,-1.943348,-1.590232,-1.00000,11.655375
50%,3.037173e+08,19.021450,0.157162,0.268929,-0.155179,11.886550,-0.031145,0.102301,1.000000,22.982100,-0.314780,0.121675,-0.917953,9.572210,-0.242269,0.048885,0.00000,19.287450
75%,4.610019e+08,45.315975,4.812810,5.320955,12.849025,17.717850,1.193810,1.626180,1.000000,62.782400,4.317110,4.727145,18.184650,17.256750,1.796523,1.689635,1.00000,34.732550
max,6.205406e+08,285.890000,57.078800,103.643000,280.571000,106.337000,2.549170,3.140730,1.000000,303.665000,108.360000,66.940900,272.391000,119.738000,2.575870,3.140660,1.00000,109.903000


Zmieniam wartości w kolumnie Q2 (dotyczącej ładunku elektronu) na 0 oznaczające, że ładunek był ujemny oraz 1 oznaczające dodatni

In [13]:
data.Q2[data.Q2 <= 0] = -1
data.Q2[data.Q2 > 0] = 1

C:\Users\Admin\AppData\Local\Temp\ipykernel_17832\869552300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Q2[data.Q2 <= 0] = -1
C:\Users\Admin\AppData\Local\Temp\ipykernel_17832\869552300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Q2[data.Q2 > 0] = 1


Naprawiam nadmiarową spację w pliku csv w datasecie

In [14]:
data.rename(columns = {'px1 ':'px1'}, inplace=True)

# Zapisuję dane w dodatkowych plikach CSV

In [15]:
data.to_csv('parseddata.csv', index=False)